In [ ]:
# print('\n\n\n### Computing inliers between initial_pair and all other images ###\n')

# min_its = 10000
# max_its = 60000
# scale_its = 3
# alpha = 0.99
# compute_inliers = True

# if compute_inliers:
    
#     for i in range(n_imgs):

#         if i != init_pair[0]:
#             print('\nImage:', i+1, '/', n_imgs)

#             x1_norm = x1s_norm_TR[i]
#             x2_norm = x2s_norm_TR[i]
#             E, inliers = estimate_E_robust(K, x1_norm, x2_norm, min_its, max_its, scale_its, alpha, pixel_threshold, essential_matrix=True, homography=True, verbose=True)

#             np.save('data/dataset_{}_TR_E_{}.npy'.format(data_set, i), E)
#             np.save('data/dataset_{}_TR_E_inliers_{}.npy'.format(data_set, i), inliers)

#             if i == init_pair[1]:
#                 # np.save('data/dataset_{}_TR_E_{}.npy'.format(data_set, init_pair[0]), E)
#                 np.save('data/dataset_{}_TR_E_inliers_{}.npy'.format(data_set, init_pair[0]), inliers)

In [ ]:
# P1 = cv.get_canonical_camera()
# best_P2 = None
# best_X = None
# max_inliers = 0
# if epsilon > epsilon_E:

#     # P2_arr = cv.extract_P_from_E(E)
#     # X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm, x2_norm)
#     # P2_valid, X_valid = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr)
#     # valid_inliers = compute_valid_inliers(P1, P2_valid, X_valid, inliers)
#     # n_valid_inliers = np.sum(valid_inliers)
#     # print(n_valid_inliers, n_inliers)

#     # best_P2 = np.copy(P2_valid)
#     # best_X = np.copy(X_valid)
#     best_E = np.copy(E)
#     best_inliers = np.copy(inliers)
#     epsilon_E = epsilon
#     print('No. inliers:', n_inliers, end='\r')

#     # if n_valid_inliers > max_inliers:
#     #     best_inliers = np.copy(valid_inliers)
#     #     best_P2 = np.copy(P2_valid)
#     #     max_inliers = n_valid_inliers
#     #     print('No. valid inliers:', n_valid_inliers, end='\r')

#         # epsilon = max_inliers / n_points
#         # T = cv.compute_ransac_iterations(alpha, epsilon, n_samples)
#         # print('New T:', T, 'New epsilon:', epsilon)
#         # if t >= 4*T-1:
#         #     print('Bailout at iteration:', t, T)
#         #     break

# valid_inliers = compute_valid_inliers(P1, best_P2, best_X, best_inliers)
# n_valid_inliers = np.sum(valid_inliers)
# print('No. valid inliers:', n_valid_inliers, 'No. inliers:', np.sum(best_inliers))


In [ ]:
def compute_sift_points_vlfeat(img1, img2, marg, verbose=False):
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY) 
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) 

    sift = cv2.xfeatures2d.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # print('opencv:', des1.shape, des2.shape)

    # kp1, des1 = vl.sift.sift(np.array(img1, dtype=np.float32), compute_descriptor=True)
    # kp2, des2 = vl.sift.sift(np.array(img2, dtype=np.float32), compute_descriptor=True)

    # print('vlfeat.', des1.shape, des2.shape)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < marg*n.distance:
            good_matches.append([m])

    # x1 = np.column_stack([kp1[match[0].queryIdx,:2] for match in good_matches])
    # x2 = np.column_stack([kp2[match[0].trainIdx,:2] for match in good_matches])
    x1 = np.column_stack([kp1[match[0].queryIdx].pt for match in good_matches])
    x2 = np.column_stack([kp2[match[0].trainIdx].pt for match in good_matches])
    x1 = cv.homogenize(x1, multi=True)
    x2 = cv.homogenize(x2, multi=True)

    des1 = np.row_stack([des1[match[0].queryIdx] for match in good_matches])
    des2 = np.row_stack([des2[match[0].trainIdx] for match in good_matches])

    if verbose:
        print('Number of matches:', np.size(matches,0))
        print('Number of good matches:', np.size(x1,1))

    return x1, x2, kp1, kp2, des1, des2

def compute_sift_points_TR_vlfeat(x1, des1, img2, marg, verbose=False):
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    sift = cv2.xfeatures2d.SIFT_create()
    kp2, des2 = sift.detectAndCompute(img2, None)
    # print('opencv:', des2.shape)

    # kp2, des2 = vl.sift.sift(np.array(img2, dtype=np.float32), compute_descriptor=True)
    # print('vlfeat.', des2.shape)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < marg*n.distance:
            good_matches.append([m])

    x_idx = np.array([match[0].queryIdx for match in good_matches])
    x1 = np.column_stack([x1[:,match[0].queryIdx] for match in good_matches])
    # x2 = np.column_stack([kp2[match[0].trainIdx,:2] for match in good_matches])
    x2 = np.column_stack([kp2[match[0].trainIdx].pt for match in good_matches])
    x2 = cv.homogenize(x2, multi=True)

    if verbose:
        print('Number of matches:', np.size(matches,0))
        print('Number of good matches:', np.size(x1,1))

    return x1, x2, x_idx

In [ ]:
print('\n\n\n### Computing and saving SIFT-points ###\n')

print('\n\n## For translation registration ##')

sift_TR = False
marg = 0.75

if sift_TR:
    
    print("\nImage:", init_pair[1]+1, "/", n_imgs, '(initial image pair)')
    x1_init, x2_init, kp1_init, kp2_init, des1_init, des2_init = compute_sift_points(img1_init, img2_init, marg, verbose=True)
    np.save('data/dataset_{}_TR_x1_{}.npy'.format(data_set, init_pair[1]), x1_init)
    np.save('data/dataset_{}_TR_x2_{}.npy'.format(data_set, init_pair[1]), x2_init)

    X_visible = np.ones(x1_init.shape[1], dtype=bool)
    np.save('data/dataset_{}_TR_X_visible_{}.npy'.format(data_set, init_pair[0]), X_visible)
    np.save('data/dataset_{}_TR_X_visible_{}.npy'.format(data_set, init_pair[1]), X_visible)

    for i in range(n_imgs):

        if i != init_pair[0] and i != init_pair[1]:

            if i > int(n_imgs/2):
                x1_init = np.copy(x2_init)
                des1_init = np.copy(des2_init)
            
            print("\nImage:", i+1, "/", n_imgs)
            img2 = imgs[i]
            x1, x2, x_idx = compute_sift_points_TR(x1_init, des1_init, img2, marg, verbose=True)
            x_idx = np.sort(x_idx)
            X_visible = []
            t = 0

            for j in range(x1_init.shape[1]):
                try:
                    if j == x_idx[t]:
                        X_visible.append(True)
                        t += 1
                    else:
                        X_visible.append(False)
                except:
                    X_visible.append(False)
            X_visible = np.array(X_visible)
                    
            np.save('data/dataset_{}_TR_X_visible_{}.npy'.format(data_set, i), X_visible)
            np.save('data/dataset_{}_TR_x1_{}.npy'.format(data_set, i), x1)
            np.save('data/dataset_{}_TR_x2_{}.npy'.format(data_set, i), x2)

In [ ]:
print('\n\n\n### Loading SIFT-points ###\n')

# SIFT points for rotation averaging
x1s_norm_RA = []
x2s_norm_RA = []
matlab_RA = False

if False:
    for i in range(n_camera_pairs):

        if matlab_RA:
            x1 = cv.convert_mat_to_np('data/dataset_{}_RA_mat_x1_{}.mat'.format(data_set, i), 'x1')
            x2 = cv.convert_mat_to_np('data/dataset_{}_RA_mat_x2_{}.mat'.format(data_set, i), 'x2')
        else:
            x1 = np.load('data/dataset_{}_RA_x1_{}.npy'.format(data_set, i))
            x2 = np.load('data/dataset_{}_RA_x2_{}.npy'.format(data_set, i))

        x1_norm = cv.dehomogenize(K_inv @ x1)
        x2_norm = cv.dehomogenize(K_inv @ x2)
        x1s_norm_RA.append(x1_norm)
        x2s_norm_RA.append(x2_norm)
        
        
    # SIFT points for translation registration
    Xs_visible_TR = []
    x1s_norm_TR = []
    x2s_norm_TR = []
    matlab_TR = False

    for i in range(n_imgs):

        if i != init_pair[0]:

            if matlab_TR:
                x1 = cv.convert_mat_to_np('data/dataset_{}_TR_mat_x1_{}.mat'.format(data_set, i), 'x1')
                x2 = cv.convert_mat_to_np('data/dataset_{}_TR_mat_x2_{}.mat'.format(data_set, i), 'x2')
            else:
                x1 = np.load('data/dataset_{}_TR_x1_{}.npy'.format(data_set, i))
                x2 = np.load('data/dataset_{}_TR_x2_{}.npy'.format(data_set, i))

            x1_norm = cv.dehomogenize(K_inv @ x1)
            x2_norm = cv.dehomogenize(K_inv @ x2)
            x1s_norm_TR.append(x1_norm)
            x2s_norm_TR.append(x2_norm)

        elif i == init_pair[0]:
            x1s_norm_TR.append(None)
            x2s_norm_TR.append(None)

        if matlab_TR:
            X_visible = cv.convert_mat_to_np('data/dataset_{}_TR_mat_X_visible_{}.mat'.format(data_set, i), 'X_visible')[0]
        else:
            X_visible = np.load('data/dataset_{}_TR_X_visible_{}.npy'.format(data_set, i))
            
        Xs_visible_TR.append(np.array(X_visible, dtype=bool))

In [ ]:
def compute_jacobian_of_residual_wrt_w1(Pi, Xj):
    
    Ri = Pi[:,:-1]
    S1 = cv.create_skew_symmetric_matrix(np.array([1,0,0]))

    J1 = ((S1 @ Ri @ Xj[:-1]) * (Pi[0,:] @ Xj) / (Pi[-1,:] @ Xj)**2) - ((S1 @ Ri @ Xj[:-1]) / (Pi[-1,:] @ Xj))
    J2 = ((S1 @ Ri @ Xj[:-1]) * (Pi[1,:] @ Xj) / (Pi[-1,:] @ Xj)**2) - ((S1 @ Ri @ Xj[:-1]) / (Pi[-1,:] @ Xj))

    J = np.row_stack((J1, J2))   

    return J

def compute_jacobian_of_residual_wrt_w2(Pi, Xj):
    
    Ri = Pi[:,:-1]
    S2 = cv.create_skew_symmetric_matrix(np.array([0,1,0]))

    J1 = ((S2 @ Ri @ Xj[:-1]) * (Pi[0,:] @ Xj) / (Pi[-1,:] @ Xj)**2) - ((S2 @ Ri @ Xj[:-1]) / (Pi[-1,:] @ Xj))
    J2 = ((S2 @ Ri @ Xj[:-1]) * (Pi[1,:] @ Xj) / (Pi[-1,:] @ Xj)**2) - ((S2 @ Ri @ Xj[:-1]) / (Pi[-1,:] @ Xj))

    J = np.row_stack((J1, J2))   

    return J

def compute_jacobian_of_residual_wrt_w3(Pi, Xj):
    
    Ri = Pi[:,:-1]
    S3 = cv.create_skew_symmetric_matrix(np.array([0,0,1]))

    J1 = ((S3 @ Ri @ Xj[:-1]) * (Pi[0,:] @ Xj) / (Pi[-1,:] @ Xj)**2) - ((S3 @ Ri @ Xj[:-1]) / (Pi[-1,:] @ Xj))
    J2 = ((S3 @ Ri @ Xj[:-1]) * (Pi[1,:] @ Xj) / (Pi[-1,:] @ Xj)**2) - ((S3 @ Ri @ Xj[:-1]) / (Pi[-1,:] @ Xj))

    J = np.row_stack((J1, J2))   

    return J

    


def compute_jacobian_of_residual_wrt_X(Pi, Xj):
    J1 = (Pi[-1,:] * (Pi[0,:] @ Xj) / (Pi[-1,:] @ Xj)**2) - (Pi[0,:] / (Pi[-1,:] @ Xj))
    J2 = (Pi[-1,:] * (Pi[1,:] @ Xj) / (Pi[-1,:] @ Xj)**2) - (Pi[1,:] / (Pi[-1,:] @ Xj))
    J = np.row_stack((J1, J2))
    return J



def compute_axis_angle_representation(R):
    
    theta = np.arccos((np.trace(R)-1)/2)
    a = (1/(2*np.sin(theta))) * np.array([R[2,1]-R[1,2], R[0,2]-R[2,0], R[1,0]-R[0,1]])
    w = theta * a

    return w

def compute_R_matrix_exponential(w, approx=False):

    wx = cv.create_skew_symmetric_matrix(w)
    w_norm = LA.norm(w)
    R = np.eye(3) + (np.sin(w_norm)*wx/w_norm) + ((1-np.cos(w_norm))*(wx @ wx)/(w_norm**2))
    if approx:
        R = np.eye(3) + wx

    return R

